<a href="https://colab.research.google.com/github/Kishan-Kumar-Zalavadia/Agentic_AI_with_Python_And_Generative_AI/blob/main/2_AIAgentsAndGenerativeAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI-Agent Tool Descriptions and Naming**

## **Describing Tools to the Agent**

When developing an agentic AI system, one of the most critical aspects is ensuring that the agent understands the tools it has access to. In our previous tutorial, we explored how an AI agent interacts with an environment. Now, we extend that discussion to focus on tool definition, particularly the importance of naming, parameters, and structured metadata.

## **Example: Automating Documentation for Python Code**

Imagine we are building an AI agent that scans through all Python files in a src/ directory and automatically generates corresponding documentation files in a docs/ directory. This agent will need to:

- List Python files in the src/ directory.
- Read the content of each Python file.
- Write documentation files in the docs/ directory.

Since file operations are straightforward for humans but ambiguous for an AI without context, we must clearly define these tools so the agent knows how to use them effectively.


# **Step 1: Defining a Tool with Structured Metadata**

A basic tool definition in Python might look like this:

```
def list_python_files():
    """Returns a list of all Python files in the src/ directory."""
    return [f for f in os.listdir("src") if f.endswith(".py")]
```

This provides a function that retrieves all Python files in the src/ directory, but for an AI system, we need a more structured way to describe it.

# **Step 2: Using JSON Schema to Define Parameters**

When developers design APIs, they use structured documentation to describe available functions, their inputs, and their outputs. JSON Schema is a well-known format for defining APIs, making it a natural choice for AI agents as well.

For example, a tool that reads a file should specify that it expects a file_path parameter of type string. JSON Schema allows us to express this in a standardized way:

```
{
  "tool_name": "read_file",
  "description": "Reads the content of a specified file.",
  "parameters": {
    "type": "object",
    "properties": {
      "file_path": { "type": "string" }
    },
    "required": ["file_path"]
  }
}
```

Similarly, a tool for writing documentation should define that it requires a file_name and content:



```
{
  "tool_name": "write_doc_file",
  "description": "Writes a documentation file to the docs/ directory.",
  "parameters": {
    "type": "object",
    "properties": {
      "file_name": { "type": "string" },
      "content": { "type": "string" }
    },
    "required": ["file_name", "content"]
  }
}
```

By providing a JSON Schema for each tool:

1. The AI can Recognize the tool’s purpose.
2. The AI / Environment interface can validate input parameters before execution.

It may look strange that multiple parameters to a function are represented as an object. When we are getting the agent to output a tool / action selection, we are going to want it to output something like this:

```
{
  "tool_name": "read_file",
  "args": {
    "file_path": "src/file.py"
  }
}
```

The schema describes the overall dictionary that will be used to capture the “args” to the function, so it is described as an object.



# Install libraries

In [ ]:
!!pip install litellm
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

# Agent that Calls Python Functions

Let's make an agent that can call Python functions.
When you run the second block in the notebook, the agent will prompt you for what action to take. You can say something like "tell me the files in the current directory" and it will make the appropriate tool choice. Experiment with the agent to see what it can and cannot do.

In [8]:
import json
import os
import sys
from litellm import completion
from typing import List, Dict

def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get a response."""
    response = completion(
        model="groq/llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content.strip()

def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

# Define system instructions (Agent Rules)
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "list_files": {
        "description": "Lists all files in the current directory.",
        "parameters": {}
    },
    "read_file": {
        "description": "Reads the content of a file.",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "The name of the file to read."
            }
        }
    },
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    }
}
```

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:
    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1


What would you like me to do? Tell me what is in the dir.
Agent thinking...
Agent response: To find out what is in the current directory, I need to list all the files. This will give me a clear understanding of the files available before I can proceed with any further actions. By listing the files, I can identify the types of documents or content available and provide a more accurate response to the user's inquiry.

```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['.config', 'Readme.txt', '.ipynb_checkpoints', 'sample_data']}
Agent thinking...
Agent response: The directory contains several files and directories, including ".config", "Readme.txt", ".ipynb_checkpoints", and "sample_data". Since the user initially asked about the contents of the directory, I should now read the "Readme.txt" file to provide more information about the directory's contents. The "Readme.txt" file often contains important information or descriptions about the directory

# **Using LLM Function Calling for AI-Agent Interaction**

# Step 3: Using LLM Function Calling for Structured Execution

One of the most challenging aspects of integrating AI agents with tool execution is ensuring that the model consistently produces structured output that can be parsed correctly. Traditionally, developers would attempt to engineer prompts to make the model output well-formed JSON, but this approach is unreliable—models can introduce variations, omit required fields, or output unstructured text that breaks parsing logic.

To solve this, most LLMs offer function calling APIs that guarantee structured execution. Instead of treating function execution as a free-form text generation task, function calling APIs allow us to explicitly define the tools available to the model using JSON Schema. The model then decides when and how to call these functions, ensuring structured and predictable responses.

## How Function Calling Simplifies AI-Agent Interfaces

When using function calling, the model returns either:

1. A function call that includes the tool name and arguments as structured JSON.
2. A standard text response if the model decides a function is unnecessary.

This approach removes the need for manual prompt engineering to enforce structured output and allows the agent to focus on decision-making rather than syntax compliance.



## Example: Executing Function Calls

Below is a typical way to invoke function calling using OpenAI’s API:



In [9]:
import json
import os
from typing import List

from litellm import completion

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"


tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]

# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

messages = agent_rules + memory

response = completion(
    model="groq/llama-3.3-70b-versatile",
    messages=messages,
    tools=tools,
    max_tokens=1024
)

# Extract the tool call from the response, note we don't have to parse now!
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
result = tool_functions[tool_name](**tool_args)

print(f"Tool Name: {tool_name}")
print(f"Tool Arguments: {tool_args}")
print(f"Result: {result}")

What would you like me to do? Give me the list of files in the directory
Tool Name: list_files
Tool Arguments: {}
Result: ['.config', 'Readme.txt', '.ipynb_checkpoints', 'sample_data']


## Breaking Down Function Calling Step by Step

Let’s examine how function calling works in detail:

## 1. Define the Tool Functions

```
def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"
```
First, we define the actual Python functions that will be executed. These contain the business logic for each tool and handle the actual operations the AI agent can perform.

## 2. Create a Function Registry

```
tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}
```

We maintain a dictionary that maps function names to their corresponding Python implementations. This registry allows us to easily look up and execute the appropriate function when the model calls it.

## 3. Define Tool Specifications Using JSON Schema
```
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]
```

This is where we describe our tools to the model. Each tool specification includes:

- A name that matches a key in our tool_functions dictionary
- A description that helps the model understand when to use this tool
- Parameters defined using JSON Schema, specifying the expected input format

Note how the list_files function takes no parameters (empty “properties” object), while read_file requires a “file_name” string parameter. The model will use these specifications to generate properly structured calls.

## 4. Set Up the Agent’s Instructions

```
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]
```

The system message provides guidance on how the agent should behave. With function calling, we don’t need to instruct the model on how to format its outputs - we only need to focus on the decision-making logic.

The system message provides guidance on how the agent should behave. With function calling, we don’t need to instruct the model on how to format its outputs - we only need to focus on the decision-making logic.

## 5. Prepare the Conversation Context

```
user_task = input("What would you like me to do? ")
memory = [{"role": "user", "content": user_task}]
messages = agent_rules + memory
```

We combine the system instructions with the user’s input to create the conversation context.

## 6. Make the API Call with Function Definitions

```
response = completion(
    model="openai/gpt-4o",
    messages=messages,
    tools=tools,
    max_tokens=1024
)
```

The critical difference here is the inclusion of the tools parameter, which tells the model what functions it can call. This is what activates the function calling mechanism.

## 7. Process the Structured Response
```
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
```

When using function calling, the response comes back with a dedicated tool_calls array rather than free-text output. This ensures that:

The function name is properly identified
The arguments are correctly formatted as valid JSON
We don’t need to parse or extract from unstructured text

## 8. Execute the Function with the Provided Arguments

```
result = tool_functions[tool_name](**tool_args)
```

Finally, we look up the appropriate function in our registry and call it with the arguments the model provided. The **tool_args syntax unpacks the JSON object into keyword arguments.



# Key Benefits of Function Calling APIs

1. **Eliminates prompt engineering for structured responses** – No need to force the model to output JSON manually.
2. **Uses standardized JSON Schema** – The same format used in API documentation applies seamlessly to AI interactions.
3. **Allows mixed text and tool execution** – The model can decide whether a tool is necessary or provide a natural response.
4. **Simplifies parsing logic** – Instead of handling inconsistent outputs, developers only check for tool_calls in the response.
5. **Guarantees syntactically correct arguments** – The model automatically ensures arguments match the expected parameter format.

# Conclusion
Function calling APIs significantly improve the reliability of AI-agent interactions by enforcing structured execution. By defining tools with JSON Schema and letting the model determine when to use them, we build a more predictable and maintainable AI environment interface. In the next section, we will explore how to register these tools dynamically using decorators to further streamline agent development.